# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import time
import csv
from PIL import Image

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import (Conv2D, MaxPooling2D, Flatten, 
                          Dense, Dropout, BatchNormalization, 
                          LeakyReLU, DepthwiseConv2D, MaxPooling2D,
                          Add)
from keras.models import model_from_json

2023-05-15 06:07:58.147641: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 06:07:58.200098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 06:07:59.840676: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os 
import logging

# Set Tensorflow log level to error 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Set logger level to ERROR
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)  # Set logger level to ERROR

# Paths

In [3]:
# Training sets
raw_train_path = "../02_Data/face_age_data/face_age_balanced_train"
aug_train_path = "../02_Data/face_age_data/augmented_data_train" 


# Validation set
raw_val_path = "../02_Data/face_age_data/face_age_balanced_val"

# Testing set
raw_test_path = "../02_Data/face_age_data/face_age_balanced_test"

# Load Datasets

In [4]:
def load_data(folders, img_size=(200, 200)):
    X = []
    y = []
    for folder_path in folders:
        for folder in os.listdir(folder_path):
            if os.path.isdir(os.path.join(folder_path, folder)):
                age = int(folder.replace("aug_", ""))
                for file in os.listdir(os.path.join(folder_path, folder)):
                    img_path = os.path.join(folder_path, folder, file)
                    img = Image.open(img_path)
                    img = img.resize(img_size)
                    img = np.array(img)
                    X.append(img)
                    y.append(age)
    return np.array(X), np.array(y)

In [5]:
img_size = (200, 200)

# Load training set
folder_paths_train = [
    raw_train_path, 
    # aug_train_path
    ]
X_train, y_train = load_data(folder_paths_train, img_size)
X_train = X_train / 255.0  # Normalize pixel values

# Load validation set
folder_paths_val = [raw_val_path]
X_val, y_val = load_data(folder_paths_val, img_size)
X_val = X_val / 255.0  # Normalize pixel values

# Load testing set
folder_paths_test = [raw_test_path]
X_test, y_test = load_data(folder_paths_test, img_size)
X_test = X_test / 255.0  # Normalize pixel values


In [6]:
print(
    f"Shape of X: {X_train.shape}", "\n",
    f"- {X_train.shape[0]}: Number of images in the dataset", "\n",
    f"- {X_train.shape[1]}: Height of each image", "\n",
    f"- {X_train.shape[2]}: Width of each image", "\n",
    f"- {X_train.shape[3]}: Number of channels of each image (Red, Green, and Blue)"
    )

Shape of X: (5598, 200, 200, 3) 
 - 5598: Number of images in the dataset 
 - 200: Height of each image 
 - 200: Width of each image 
 - 3: Number of channels of each image (Red, Green, and Blue)


In [7]:
print(f"Labels: {y_train}")

Labels: [32 32 32 ... 58 58 58]


In [8]:
print(
    f"Size of arrays:", "\n",
    f"- X_train shape: {X_train.shape}", "\n",
    f"- y_train shape: {y_train.shape}", "\n",
    f"- X_val shape: {X_val.shape}", "\n",
    f"- y_val shape: {y_val.shape}", "\n",
    f"- X_test shape: {X_test.shape}", "\n",
    f"- y_test shape: {y_test.shape}", "\n",
    )

Size of arrays: 
 - X_train shape: (5598, 200, 200, 3) 
 - y_train shape: (5598,) 
 - X_val shape: (1400, 200, 200, 3) 
 - y_val shape: (1400,) 
 - X_test shape: (1750, 200, 200, 3) 
 - y_test shape: (1750,) 



# Model Setup

#### Check for GPUs

In [9]:
# Check if GPUs are available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Get list of GPUS
gpus = tf.config.experimental.list_physical_devices('GPU')

# Extract names
gpu_names = ['/device:GPU:' + gpu.name.split(':')[-1] for gpu in gpus]

print(gpu_names)

Num GPUs Available:  8
['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


#### Set up training

In [10]:
# Training parameters
epochs = 1000
batch_size = 32
optimizer = "adam"
loss = "mean_squared_error"
metrics = ['mae']

In [11]:
# Create a MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=gpu_names)

def create_model(input_shape):
    model = Sequential([
        Conv2D(64, (7, 7), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (5, 5), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(256, (3, 3), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(512, (3, 3), padding='same'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1)
    ])

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

# Open the strategy scope
with strategy.scope():
    input_shape = (img_size[0], img_size[1], 3)
    model = create_model(input_shape)

2023-05-15 06:08:36.649434: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9599 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-05-15 06:08:36.650599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9599 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2023-05-15 06:08:36.651576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9599 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5
2023-05-15 06:08:36.652556: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9599 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INF

#### Train model

In [12]:
# Create a callback to save the model's weights
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("../04_Age_Prediction/08_models/best_model.h5", save_best_only=True)

# Create a callback that stops the training if there is no improvement in the validation loss for 15 consecutive epochs
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)

In [13]:
# Train the model with your data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=epochs, batch_size=batch_size, verbose=1,
                    callbacks=[checkpoint_cb, early_stopping_cb])

2023-05-15 06:08:45.690678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [5598,200,200,3]
	 [[{{node Placeholder/_10}}]]
2023-05-15 06:08:45.691019: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [5598,200,200,3]
	 [[{{node Placeholder/_10}}]]


Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 22 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 22 all-reduces with algorithm = nccl, num_packs = 1


2023-05-15 06:09:08.925064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:10.389405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:11.971801: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:13.800170: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:15.346383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:17.118627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:18.460415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:19.465516: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-15 06:09:26.456768: I tensorflow/compiler/xla/se

175/175 [==============================] - ETA: 0s - loss: 1469.6007 - mae: 31.4687

2023-05-15 06:09:44.032333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [1750]
	 [[{{node Placeholder/_11}}]]
2023-05-15 06:09:44.032654: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [1750]
	 [[{{node Placeholder/_11}}]]


175/175 [==============================] - 63s 117ms/step - loss: 1469.6007 - mae: 31.4687 - val_loss: 850.2231 - val_mae: 22.0240
Epoch 2/1000
175/175 [==============================] - 16s 92ms/step - loss: 1060.4071 - mae: 27.1807 - val_loss: 588.8397 - val_mae: 19.2267
Epoch 3/1000
175/175 [==============================] - 15s 83ms/step - loss: 671.5253 - mae: 20.7477 - val_loss: 623.0220 - val_mae: 20.5943
Epoch 4/1000
175/175 [==============================] - 16s 91ms/step - loss: 428.4073 - mae: 15.9952 - val_loss: 208.4856 - val_mae: 10.9223
Epoch 5/1000
175/175 [==============================] - 14s 83ms/step - loss: 328.8006 - mae: 14.0338 - val_loss: 642.2393 - val_mae: 20.6211
Epoch 6/1000
175/175 [==============================] - 16s 91ms/step - loss: 311.0238 - mae: 13.7697 - val_loss: 164.6209 - val_mae: 9.7810
Epoch 7/1000
175/175 [==============================] - 14s 83ms/step - loss: 290.3718 - mae: 13.1935 - val_loss: 208.7930 - val_mae: 10.6692
Epoch 8/1000
175/

In [14]:
# model = tf.keras.models.load_model('best_model.h5')

# Save, Load, and Display Model History

In [15]:
# Model attributes
timestamp = int(time.time())
file_name = '../04_Age_Prediction/model_history.csv'
description = f"{timestamp}"
input_shape = str(X_train.shape[1:])
num_params = model.count_params()
epochs = len(history.history['loss'])
optimizer = optimizer
loss_function = loss
train_mae = min(history.history['mae'])
validation_mae = min(history.history['val_mae'])

In [16]:
def append_model_history_to_file(file_name, description, input_shape, num_params, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae):
    train_mae = round(train_mae, 6)
    validation_mae = round(validation_mae, 6)

    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Model', 'Input Shape', 'Model Params', 'Epochs', 'Batch Size', 'Optimizer', 'Loss Function', 'Train MAE', 'Validation MAE'])

    with open(file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        try:
            next(csv_reader)  # Skip the header row
        except StopIteration:
            pass  # The CSV file is empty or only contains the header row

        for row in csv_reader:
            if list(map(str, row[1:])) == list(map(str, [input_shape, num_params, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])):
                print("Entry with the same parameters already exists.")
                return False

    with open(file_name, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([description, input_shape, num_params, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])

    return True

new_entry_added = append_model_history_to_file(file_name, description, input_shape, num_params, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae)

# Save model
def save_model_architecture(model, file_name):
    model_json = model.to_json()
    with open(file_name, "w") as json_file:
        json_file.write(model_json)

if new_entry_added:
    model_file = f"../04_Age_Prediction/08_models/{description}.h5"
    model.save(model_file)
    print(f"Model saved at {model_file}")
else:
    print("Model not saved as an entry with the same parameters already exists.")

Model saved at ../04_Age_Prediction/08_models/1684133411.h5


In [17]:
def load_data_to_dataframe(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None
    
    df = pd.read_csv(file_name)
    return df

file_name = '../04_Age_Prediction/model_history.csv'
df = load_data_to_dataframe(file_name)

df

,Model,Input Shape,Model Params,Epochs,Batch Size,Optimizer,Loss Function,Train MAE,Validation MAE
0,1684111562,"(200, 200, 3)",8770497,155,32,adam,mean_squared_error,12.435976,8.629502
1,1684118998,"(200, 200, 3)",33625217,132,32,adam,mean_squared_error,10.202019,6.590096
2,1684126499,"(200, 200, 3)",18144705,170,32,adam,mean_squared_error,9.910834,6.491637
3,1684130813,"(200, 200, 3)",9872321,213,32,adam,mean_squared_error,10.675910,7.109921
4,1684133411,"(200, 200, 3)",10344897,168,32,adam,mean_squared_error,10.852635,7.511125


In [18]:
# # Specify model to load
# model_to_load = "1683270923"

# def load_and_display_model_architecture(file_name):
#     if not os.path.exists(file_name):
#         print("File does not exist.")
#         return None

#     with open(file_name, "r") as json_file:
#         model_json = json_file.read()
    
#     model = model_from_json(model_json)
#     model.summary()

#     return model

# model_architecture_file = f"../04_Age_Prediction/models/{model_to_load}.json"
# model = load_and_display_model_architecture(model_architecture_file)

# 